## Updated Noise Estimation Brute Force

For a single noise value, have multiple simulations for that and that will show the distribution of possible values at that noise level
-  Note that each noise value samples from a distribution already, and the same noise value is going to have different results each time due to this
- Use MonteCarlo from monte_carlo.py
- In 2024PHMTutorial, see monte_carlo example

### Next steps
- Optimize the noise to get closer to exact

In [ ]:
from progpy.models import ThrownObject
import numpy as np
import matplotlib.pyplot as plt
from progpy.predictors import MonteCarlo
from scipy.optimize import minimize

# Pick a process and measurement noise as a goal --> then use that as "real" data
PROCESS_NOISE = 1.8
MEASUREMENT_NOISE = 0.4

m = ThrownObject(process_noise=PROCESS_NOISE, measurement_noise=MEASUREMENT_NOISE)
simulated_results = m.simulate_to(8, save_freq=1)

times = simulated_results.times
inputs = simulated_results.inputs
outputs = simulated_results.outputs

# Like estimate_params
PROCESS_NOISE_GUESS = 3
MEASUREMENT_NOISE_GUESS = 2

m = ThrownObject(process_noise=PROCESS_NOISE_GUESS, measurement_noise=MEASUREMENT_NOISE_GUESS)
initial_state = m.initialize()

# MC simulations
def future_loading(t, x=None):
    return {}

STEP_SIZE = 1
NUM_SAMPLES = 100
PREDICTION_HORIZON = times[-1]
SAVE_FREQ = 1

mc = MonteCarlo(m)
mc_results = mc.predict(initial_state, future_loading, dt=STEP_SIZE, horizon=PREDICTION_HORIZON, save_freq=SAVE_FREQ, n_samples=NUM_SAMPLES)
mc_results.states

In [ ]:
for i in range(NUM_SAMPLES):
    data_pos = [state['x'] for state in mc_results.states[i]]
    plt.plot(mc_results.times[:len(data_pos)], data_pos)

# Plot model with no noise
m = ThrownObject(process_noise=0, measurement_noise=0)
simulated_results = m.simulate_to(times[-1], save_freq=1)
no_noise_pos = [state['x'] for state in simulated_results.outputs]
plt.plot(times, no_noise_pos, color="black", linestyle="--", label="No noise simulation")

# Plot real data
pos = [state['x'] for state in outputs]
plt.plot(times, pos, color="blue", linestyle="--", label="Real data")

# Plot MC simulation mean
mc_mean_pos = [state['x'] for state in mc_results.states.mean]
plt.plot(times, mc_mean_pos, color="red", linestyle="--", label="Mean MC simulation")

error = np.mean((np.array(pos) - np.array(mc_mean_pos))**2)
print("MSE error: ", error)

plt.legend()
plt.title("Monte Carlo Simulation for one noise value of Thrown Object")
plt.xlabel("Time (s)")
plt.ylabel("Position (m)")
plt.show()

In [ ]:
mc_results.states.mean

How to optimize since there's so many simulations for only one process noise / measurement noise value?

In [ ]:
# pos = [state['x'] for state in outputs]

# def optimization_fcn(params):
#     PROCESS_NOISE_GUESS = params[0]
#     MEASUREMENT_NOISE_GUESS = params[1]

#     m = ThrownObject(process_noise=PROCESS_NOISE_GUESS, measurement_noise=MEASUREMENT_NOISE_GUESS)
#     initial_state = m.initialize()

#     # MC simulations
#     def future_loading(t, x=None):
#         return {}

#     STEP_SIZE = 1
#     NUM_SAMPLES = 100
#     PREDICTION_HORIZON = times[-1]
#     SAVE_FREQ = 1

#     mc = MonteCarlo(m)
#     mc_results = mc.predict(initial_state, future_loading, dt=STEP_SIZE, horizon=PREDICTION_HORIZON, save_freq=SAVE_FREQ, n_samples=NUM_SAMPLES)
#     mc_mean_pos = [state['x'] for state in mc_results.states.mean]

#     error = np.mean((np.array(pos) - np.array(mc_mean_pos))**2)
#     print(f"Error: {error}, PROCESS_NOISE_GUESS: {PROCESS_NOISE_GUESS}, MEASUREMENT_NOISE_GUESS: {MEASUREMENT_NOISE_GUESS}")

#     return error

# initial_guess = [1.3, 1.3]
# res = minimize(optimization_fcn, initial_guess, bounds=((0, 3), (0, 3)))

# if res.success:
#     print("Optimization converged successfully.")
# else:
#     print("Optimization did not converge.")

# optimized_process_noise = res.x[0]
# optimized_measurement_noise = res.x[1]
# print("Optimized process noise: ", optimized_process_noise)
# print("Optimized measurement noise: ", optimized_measurement_noise)